In [1]:
import json

In [2]:
with open ('results/wtq/tf_examples/random-split-1-dev_errors.json') as f:
    errors = json.load(f)

In [3]:
s=set()
t = 0
for k,v in errors.items():
    if k=='Invalid answer':
        s|=set(v) 
    print(k,len(v))
    t+=len(v)
print('total',t)
print(len(s))

Couldn't find all answers 98
Invalid answer 250
Too many rows 15
total 363
250


In [4]:
# Invalid Answers Breakdown
with open('results/wtq/interactions/err_ids.json') as f:
    bkdns = json.load(f)

In [5]:
se= set()
split = 'random-split-1-dev.tsv'
for k,v in bkdns[split].items():
    v = list(map(lambda x:x.split('_')[0],v))
    bkdns[split][k] = v
    print(k)
    print(len(v))
    se|=set(v)
print(f'{split} total',len(se))

Cannot parse answer: [float_value: Contains digits, but Unable to convert value to float]
75
Cannot parse answer: [float_value: Cannot convert to multiple answers to single float]
7
Cannot parse answer: [float_value: Unable to convert value to float]
169
Cannot parse answer: [answer_coordinates: Assignment is ambiguous][float_value: Cannot convert to multiple answers to single float]
1
random-split-1-dev.tsv total 252


In [6]:
err_key = "Cannot parse answer: [float_value: Unable to convert value to float]"
maj_err = set(bkdns['random-split-1-dev.tsv'][err_key])

In [157]:
from tapas.protos import interaction_pb2
import tensorflow.compat.v1 as tf
from google.protobuf.json_format import MessageToDict
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
todo = []
split = 'random-split-1-dev'
corr = []

for value in tf.python_io.tf_record_iterator(f'results/wtq/interactions/{split}.tfrecord'):
    interaction = interaction_pb2.Interaction()
    interaction.ParseFromString(value)
    if interaction.id in maj_err:
        todo.append(interaction)
    elif len(corr)<3:
        corr.append(interaction)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [9]:
from tapas.utils import tf_example_utils
from tapas.utils import pruning_utils
token_selector = pruning_utils.HeuristicExactMatchTokenSelector(
    'tapas_wtq_wikisql_sqa_inter_masklm_large_reset/vocab.txt', 
    512, 
    pruning_utils.SelectionType.COLUMN,
    # Only relevant for SQA where questions come in sequence
    use_previous_answer=True,
    use_previous_questions=True,
  )

In [10]:
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file='tapas_wtq_wikisql_sqa_inter_masklm_large_reset/vocab.txt', 
    max_seq_length=512, 
    max_column_id=512, 
    max_row_id=512, 
    strip_column_names=False, 
    cell_trim_length=-1, 
    add_aggregation_candidates=False, 
    expand_entity_descriptions=False, 
    use_entity_title=False, 
    entity_descriptions_sentence_limit=5, 
    use_document_title=False, 
    update_answer_coordinates=False, 
    drop_rows_to_fit=False, 
    use_context_title=False, 
    trim_question_ids=False, 
    label_sampling_rate={}
)

In [11]:
converter = tf_example_utils.ToClassifierTensorflowExample(config)

In [161]:
examples = []
#for ex in corr:
for ex in todo:
    ex = token_selector.annotated_interaction(ex)
    if len(ex.questions[0].text)>0:
        print(ex.questions[0].text)
    examples.append(converter.convert(ex,0))

In [162]:
err_out = 'results/wtq/tf_examples/rs-1-dev-no-digits.tfrecord'
with tf.io.TFRecordWriter(err_out,options='GZIP') as writer:
    for example in examples:
        writer.write(example.SerializeToString())

In [145]:
from tapas.models import tapas_classifier_model
from tapas.models.bert import modeling
from tapas.utils import tasks, hparam_utils
from tapas.utils import text_utils
import functools

task = tasks.Task.WTQ
hparams = hparam_utils.get_hparams(task)

In [159]:
predict_input_fn = functools.partial(
    tapas_classifier_model.input_fn,
    name='predict',
    file_patterns=err_out,
    data_format='tfrecord',
    compression_type='GZIP',
    is_training=False,
    max_seq_length=512,
    max_predictions_per_seq=20,
    add_aggregation_function_id=True,
    add_classification_labels=False,
    add_answer=True,
    include_id=False)

In [147]:
bert_config_file = 'tapas_wtq_wikisql_sqa_inter_masklm_large_reset/bert_config.json'
bert_config = modeling.BertConfig.from_json_file(bert_config_file)

In [148]:
tapas_config = tapas_classifier_model.TapasClassifierConfig(
    bert_config= bert_config, 
    init_checkpoint='tapas_wtq_wikisql_sqa_inter_masklm_large_reset/model.ckpt', 
    learning_rate=1.93581e-05, 
    num_train_steps=50000, 
    num_warmup_steps=6448, 
    use_tpu=False, 
    positive_weight=10.0, 
    num_aggregation_labels=4, 
    num_classification_labels=0, 
    aggregation_loss_importance=1.0, 
    use_answer_as_supervision=True, 
    answer_loss_importance=1.0, 
    use_normalized_answer_loss=False, 
    huber_loss_delta=0.121194, 
    temperature=0.0352513, 
    agg_temperature=1.0, 
    use_gumbel_for_cells=False, 
    use_gumbel_for_agg=False, 
    average_approximation_function=tapas_classifier_model.AverageApproximationFunction.RATIO, 
    cell_select_pref=0.207951, 
    answer_loss_cutoff=0.664694, 
    grad_clipping=10.0, 
    max_num_rows=64, 
    max_num_columns=32, 
    average_logits_per_cell=False, 
    select_one_column=True, 
    allow_empty_column_selection=False, 
    disabled_features=[], 
    init_cell_selection_weights_to_zero=True, 
    disable_position_embeddings=False, 
    reset_position_index_per_cell=False, 
    disable_per_token_loss=False, 
    span_prediction=tapas_classifier_model.SpanPredictionMode(
          hparams.get('span_prediction',
                      tapas_classifier_model.SpanPredictionMode.NONE)),
    proj_value_length=None,
    reset_output_cls=False,
    classification_label_weight=None,
    mask_examples_without_labels=False,
    table_pruning_config_file=None)

In [149]:
model_fn = tapas_classifier_model.model_fn_builder(tapas_config)

In [150]:
run_config = tf.estimator.tpu.RunConfig(
      cluster=None,
      master=None,
      model_dir='results/wtq/model',
      tf_random_seed=None,
      save_checkpoints_steps=1000,
      keep_checkpoint_max=5,
      keep_checkpoint_every_n_hours=4.0,
      tpu_config=tf.estimator.tpu.TPUConfig(
          iterations_per_loop=1000,
          num_shards=8,
          per_host_input_for_training=3))

In [151]:
estimator = tf.estimator.tpu.TPUEstimator(
      params={'gradient_accumulation_steps': 1},
      use_tpu=False,
      model_fn=model_fn,
      config=run_config,
      train_batch_size=512 // 1,
      eval_batch_size=None,
      predict_batch_size=32)

INFO:tensorflow:Using config: {'_model_dir': 'results/wtq/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 4.0, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_inf

In [152]:
checkpoint = estimator.latest_checkpoint()

In [163]:
result = estimator.predict(input_fn=predict_input_fn)

In [165]:
from tapas.experiments import prediction_utils as exp_prediction_utils
prediction_file = 'results/wtq/model/err_predictions.tsv'

In [166]:
exp_prediction_utils.write_predictions(
    result,
    prediction_file,
    do_model_aggregation=True,
    do_model_classification=False,
    cell_classification_threshold=0.5,
    output_token_probabilities=False,
    output_token_answers=True)

> /workspace/hsiehcc/tapas/tapas/models/tapas_classifier_model.py(1192)input_fn()
   1188       compression_type=compression_type,
   1189       is_training=is_training,
   1190       params=params)
   1191 #  import pdb;pdb.set_trace()
-> 1192   return ds



ipdb>  c


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU/GPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = aggregation_function_id, shape = (?, 1)
INFO:tensorflow:  name = answer, shape = (?, 1)
INFO:tensorflow:  name = column_ids, shape = (?, 512)
INFO:tensorflow:  name = column_ranks, shape = (?, 512)
INFO:tensorflow:  name = input_ids, shape = (?, 512)
INFO:tensorflow:  name = input_mask, shape = (?, 512)
INFO:tensorflow:  name = inv_column_ranks, shape = (?, 512)
INFO:tensorflow:  name = label_ids, shape = (?, 512)
INFO:tensorflow:  name = numeric_relations, shape = (?, 512)
INFO:tensorflow:  name = numeric_values, shape = (?, 512)
INFO:tensorflow:  name = numeric_values_scale, shape = (?, 512)
INFO:tensorflow:  name = prev_label_ids, shape = (?, 512)
INFO:tensorflow:  name = question_id_ints, shape = (?, 64)
INFO:tensorflow:  name = row_ids, shape = (?, 512)
INFO:tensorflow:  name = segment_ids, shape = (?, 512)
INFO:tensorflow:**** Trainable Variables **

In [172]:
import csv
import pandas as pd

In [173]:
def _load_predictions(path):
    with open(path) as f:
        reader = csv.reader(f, delimiter='\t', quotechar='"',quoting=csv.QUOTE_ALL,escapechar='\\')
        headers = next(reader)
        table = pd.DataFrame(list(reader),columns=headers)
    return table

In [174]:
table = _load_predictions(prediction_file)

python tapas/scripts/convert_predictions.py --interaction_files results/wtq/interactions/random-split-1-dev.tfrecord --prediction_files results/wtq/model/err_predictions.tsv --output_directory formatted_predictions --dataset_format WIKITABLEQUESTIONS

In [181]:
table.iloc[0]

question_id                                                nt-10170-0_0
id                                                             nt-10170
annotator                                                             0
position                                                              0
answer_coordinates    ['(0, 0)', '(1, 0)', '(2, 0)', '(3, 0)', '(4, ...
answers               [{"column_index": 0, "row_index": 0, "begin_to...
gold_aggr                                                             0
pred_aggr                                                             3
Name: 0, dtype: object